In [5]:
import os
import re
import json
import requests
from datetime import datetime
from typing import Any
from collections import defaultdict

from bs4 import BeautifulSoup, Tag
from urllib.parse import urljoin, unquote

BASE_URL = 'https://sv.wikipedia.org'
MAIN_URL = BASE_URL + '/wiki/Wikipedia:Visste_du_att'

OUTPUT_DIR = 'data/sv'

DEFAULT_SECTION_MONTH = 'Januari'


def get_year_links_from_archive(main_page_url: str) -> dict[str, dict[str, Any]]:
    """
    Get the year links from the main archive page.
    Current year link is the main page.
    """
    resp = requests.get(main_page_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')

    table = soup.find('td', class_='navbox-list navbox-odd')
    if not table:
        raise ValueError("Could not find the table.")

    archive_links = {}
    for a_tag in table.find_all('a', href=True):        
        year = a_tag.text.strip()
        if re.match(r'^\d{4}$', year):
            href = a_tag['href']
            full_url = urljoin(BASE_URL, href)
            exists = 'new' not in a_tag.get('class', [])
            archive_links[year] = {
                'url': full_url,
                'exists': exists
            }

    # Current year link is the main page.
    current_year = str(datetime.now().year)
    archive_links[current_year] = {
        'url': main_page_url,
        'exists': True,
    }

    return archive_links


In [2]:
archive = get_year_links_from_archive(MAIN_URL)

archive

{'2004': {'url': 'https://sv.wikipedia.org/wiki/Wikipedia:Visste_du_att.../2004',
  'exists': True},
 '2005': {'url': 'https://sv.wikipedia.org/wiki/Wikipedia:Visste_du_att.../2005',
  'exists': True},
 '2006': {'url': 'https://sv.wikipedia.org/wiki/Wikipedia:Visste_du_att.../2006',
  'exists': True},
 '2007': {'url': 'https://sv.wikipedia.org/wiki/Wikipedia:Visste_du_att.../2007',
  'exists': True},
 '2008': {'url': 'https://sv.wikipedia.org/wiki/Wikipedia:Visste_du_att.../2008',
  'exists': True},
 '2009': {'url': 'https://sv.wikipedia.org/wiki/Wikipedia:Visste_du_att.../2009',
  'exists': True},
 '2010': {'url': 'https://sv.wikipedia.org/wiki/Wikipedia:Visste_du_att.../2010',
  'exists': True},
 '2011': {'url': 'https://sv.wikipedia.org/wiki/Wikipedia:Visste_du_att.../2011',
  'exists': True},
 '2012': {'url': 'https://sv.wikipedia.org/wiki/Wikipedia:Visste_du_att.../2012',
  'exists': True},
 '2013': {'url': 'https://sv.wikipedia.org/wiki/Wikipedia:Visste_du_att.../2013',
  'exists

In [49]:
def _extract_fact_data(element: Tag, base_url: str) -> dict[str, Any]:
    """Extracts text, links, and relevant links from a BeautifulSoup Tag."""
    fact_text = "Visste du att " + element.get_text(" ", strip=True).lstrip().lstrip('.').lstrip('…').lstrip()
    fact_text = fact_text.replace('(se bild)', '').replace('( se bild )', '').replace('  ', ' ')
    fact_text = fact_text.replace('\xa0', ' ')

    links = []
    relevant_links = []
    for a in element.find_all("a", href=True):
        href = a["href"]
        if not href.startswith("/wiki/"):
            continue

        full_url = unquote(urljoin(base_url, href))

        if a.find_parent('b'):
            relevant_links.append(full_url)

        links.append(full_url)

    return {
        "text": fact_text,
        "links": links,
        "relevant_links": relevant_links,
    }


def parse_year_facts(year_url: str, year: str) -> list[dict]:
    """Parse the facts from the year page."""
    resp = requests.get(year_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    results: list[dict] = []

    # Parse facts from all uls in section div
    div = soup.find("div", class_="mw-content-ltr mw-parser-output")
    if not div:
        return []
    
    # Different parsing for different years.
    if int(year) <= 2010: 
        ps = div.find_all("p", recursive=False)

        for p in ps:
            html_content = p.decode_contents()
            
            # Split by <br> tags, handling multiple separators and surrounding whitespace.
            fact_htmls = re.split(r'(?:<br[^>]*>\s*)+', html_content)
            
            # Filter out empty strings that may result from leading/trailing <br>s
            fact_htmls = [html.strip() for html in fact_htmls if html.strip()]

            for fact_html in fact_htmls:
                # Create a temporary soup to parse the fact snippet
                fact_soup = BeautifulSoup(f"<span>{fact_html}</span>", "html.parser").span
                if not fact_soup:
                    continue
                
                fact_data = _extract_fact_data(fact_soup, year_url)
                section = DEFAULT_SECTION_MONTH + ' ' + year

                if fact_data['text']:
                    results.append({
                        "section": section,
                        **fact_data
                    })
    elif int(year) <= 2012:
        for elem in div:
            if not (isinstance(elem, Tag) and elem.name == "ul"):
                continue

            for li in elem.find_all("li"):
                fact_data = _extract_fact_data(li, year_url)
                section = DEFAULT_SECTION_MONTH + ' ' + year

                if fact_data['text']:
                    results.append({
                        "section": section,
                        **fact_data
                    })
    else:
        current_section_name = DEFAULT_SECTION_MONTH + ' ' + year
        for elem in div:
            if not (isinstance(elem, Tag) and (elem.name == 'div' or elem.name == 'ul')):
                continue
            
            # End of facts
            if (
                elem.name == 'div' and 
                'mw-heading' in elem.get('class', []) and 
                'mw-heading2' in elem.get('class', []) and 
                'ext-discussiontools-init-section' in elem.get('class', [])
            ):
                h2 = elem.find('h2')
                if h2 and h2.text.strip() == 'Se även':
                    break
        
            # New section
            if elem.name == 'div' and 'mw-heading3' in elem.get('class', []):
                h3 = elem.find('h3')
                if h3:
                    current_section_name = h3.text.strip()

            # Fact
            elif elem.name == 'ul':
                for li in elem.find_all('li'):
                    fact_data = _extract_fact_data(li, year_url)

                    if fact_data['text']:
                        results.append({
                            "section": current_section_name,
                            **fact_data
                        })

    return results


In [56]:
year = '2025'

facts = parse_year_facts(archive[year]['url'], year)
facts

[{'section': 'Juli 2025',
  'text': 'Visste du att melodin till den engelska bröllopsmarschen "Here Comes the Bride" kommer från " Brudkören " ur Wagners opera Lohengrin ?',
  'links': ['https://sv.wikipedia.org/wiki/Brudkören',
   'https://sv.wikipedia.org/wiki/Richard_Wagner',
   'https://sv.wikipedia.org/wiki/Lohengrin_(opera)'],
  'relevant_links': ['https://sv.wikipedia.org/wiki/Brudkören',
   'https://sv.wikipedia.org/wiki/Lohengrin_(opera)']},
 {'section': 'Juli 2025',
  'text': 'Visste du att judarnas historia i Iran är över 2 700 år gammal?',
  'links': ['https://sv.wikipedia.org/wiki/Judarnas_historia_i_Iran'],
  'relevant_links': ['https://sv.wikipedia.org/wiki/Judarnas_historia_i_Iran']},
 {'section': 'Juli 2025',
  'text': 'Visste du att Emma Beckers La Maison är en autofiktiv roman om hennes två år som sexarbetare på en bordell i Berlin ?',
  'links': ['https://sv.wikipedia.org/wiki/Emma_Becker',
   'https://sv.wikipedia.org/wiki/La_Maison',
   'https://sv.wikipedia.org/w

In [53]:
MONTHS = {
    'січень': 'січень', 'січня': 'січень',
    'лютий': 'лютий', 'лютого': 'лютий',
    'березень': 'березень', 'березня': 'березень',
    'квітень': 'квітень', 'квітня': 'квітень',
    'травень': 'травень', 'травня': 'травень',
    'червень': 'червень', 'червня': 'червень',
    'липень': 'липень', 'липня': 'липень',
    'серпень': 'серпень', 'серпня': 'серпень',
    'вересень': 'вересень', 'вересня': 'вересень',
    'жовтень': 'жовтень', 'жовтня': 'жовтень',
    'листопад': 'листопад', 'листопада': 'листопад',
    'грудень': 'грудень', 'грудня': 'грудень',
}

def extract_month_from_title(title: str) -> str | None:
    """Extracts the first mentioned month from a section title."""
    words = re.split(r'[\s—-]+', title)

    for word in words:
        cleaned_word = word.lower().strip()
        
        if cleaned_word in MONTHS:
            return MONTHS[cleaned_word]

    return None

In [60]:
extract_month_from_title('3 22 грудня 2022 — 23 січня 2023')

'грудень'

In [52]:
import dateparser

year = '2025'
month = '7'
section = '11 — 25 лютого 2025'

dt_month = dateparser.parse(section)
fact_date = dt_month.strftime("%Y-%m") if dt_month else None

fact_date

'2025-02'